In [ ]:
!pip install langchain langchain-community chromadb pypdf sentence-transformers openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 128.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 7.3 MB/s eta 

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
# get the api key from below website
# https://openrouter.ai/

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

#from langchain_community.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

import os
from google.colab import userdata
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Get OpenRouter API key and set it as an environment variable
os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")

# Step 1: Set up the knowledge base
# Convert the list of strings (chunks) into a list of Document objects
loader=PyPDFLoader("/content/Gabriel_AnnualReport.pdf")
pages=loader.load_and_split()
pdf_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
docs=pdf_splitter.split_documents(pages)
documents = [Document(page_content=doc.page_content) for doc in docs]

# Initialize embeddings using HuggingFaceEmbeddings with OpenRouter endpoint
# You might need to specify the model name if OpenRouter requires it
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2", # Example model, replace if needed
    model_kwargs={'device': 'cpu'}, # Or 'cuda' if you have a GPU
    encode_kwargs={'normalize_embeddings': False},
    # There isn't a direct 'base_url' parameter for HuggingFaceEmbeddings for custom APIs.
    # We will rely on OpenRouter's API key to route the request.
    # If this doesn't work, we might need a custom embedding class or a different approach.
)


vector_db = Chroma.from_documents(
    documents,  # Use the list of Document objects
    embedding=embeddings
)

# Step 2: Set up conversational memory
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True
)

# Step 3: Create the LLM
# Initialize ChatOpenAI with the OpenRouter base URL and desired model, passing the OpenRouter API key
llm = ChatOpenAI(
    model="openai/gpt-3.5-turbo",
    temperature=0.2,
    openai_api_base="https://openrouter.ai/api/v1",
    max_tokens=500,
    openai_api_key=os.environ["OPENROUTER_API_KEY"] # Pass the OpenRouter API key here
)

# Step 4: Combine retriever and LLM into a chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_db.as_retriever(),
    memory=memory
)

# Step 5: Real-time interaction loop
while True:
    question = input("User: ")
    answer = qa_chain({"question": question})
    if question == "Exit":
        print("Thank You, See You later!!!")
        break
    print("Bot:", answer["answer"])

/tmp/ipython-input-2220505181.py:56: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


User: what is gabriel company about?


/tmp/ipython-input-2220505181.py:74: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain({"question": question})


Bot: Gabriel India is a company that specializes in suspension solutions. They are focused on market-driven product strategies, global alliances, and compliance-led excellence to unlock international growth. They are expanding their product range and operating across multiple segments to serve a broader customer base with high-quality components and systems. Additionally, Gabriel India is undergoing a major corporate restructuring to become the primary growth engine of the ANAND Group, with a target of achieving significant growth by 2030. Their people-first philosophy has also gained external recognition, with the company being honored as a Workplace of Winner (WoW) - Best Place to Work.
User: does anand and gabriel cpmapny diffrent?
Bot: ANAND and Gabriel India Limited are different entities. ANAND Group is the parent company, and Gabriel India Limited is one of the companies within the ANAND Group. Gabriel India Limited is the flagship company of the ANAND Group, which means it is a